In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from epiweeks import Week
from datetime import date, timedelta
from os.path import join

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia
from bulletin.systems.casos_confirmados import CasosConfirmados

In [ ]:
cc = CasosConfirmados()
#cc.databases()
cc.load(f"cc_{str(date.today().day-1).zfill(2)}_{str(date.today().month).zfill(2)}_{str(date.today().year).zfill(4)}", compress=True)

In [ ]:
# casos_confirmados = CasosConfirmados(force=True, hard=True)
# casos_confirmados.shape()

In [ ]:
#Usar 0 para hoje e 1 para ontem
# algarismo = 1
# data = datetime.today() - timedelta(algarismo)
# dia = data.day
# mes = data.month
# ano = data.year

# casos = pd.read_csv(f"input/informe_epidemiologico_{str(dia).zfill(2)}_{str(mes).zfill(2)}_{ano}_geral.csv",
#                     sep = ';',
#                     converters = {
#                         'DATA_DIAGNOSTICO' : pd.to_datetime,
#                         'DATA_INICIO_SINTOMAS' : pd.to_datetime
#                     }
#                    )
# casos = casos_confirmados.get_casos()

In [ ]:
casos = cc.df.loc[((cc.df['uf_residencia'] == 'PR')) & (cc.df['classificacao_final'] == 2)].copy()
# casos.loc[(casos['OBITO'] == 'SIM') & (~(casos['IBGE_RES_PR'] == 9999999))]

In [ ]:
del cc

In [ ]:
sivep = pd.read_excel(join(default_input,'TABELA_sivep_INPUT.xlsx'))
notifica = pd.read_excel(join(default_input,'TABELA_notifica_INPUT.xlsx'))
# sivep = sivep.rename(columns={'Unnamed: 0':'SE'})
# notifica = notifica.rename(columns={'Unnamed: 0':'SE'})
notifica = notifica.set_index('SE')
sivep = sivep.set_index('SE')

In [ ]:
# casos['DATA_DIAGNOSTICO'] = casos['DATA_DIAGNOSTICO'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce'))
# casos['DATA_INICIO_SINTOMAS'] = casos['DATA_INICIO_SINTOMAS'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce'))

In [ ]:
casos['se_diag'] = casos['data_diagnostico'].apply(lambda x: Week.fromdate(x))
casos.loc[~casos['data_1o_sintomas'].isnull(), 'se_is'] = casos['data_1o_sintomas'].loc[~casos['data_1o_sintomas'].isnull()].apply(lambda x: Week.fromdate(x))

In [ ]:
tabela_diag = casos.groupby(['se_diag'])[['data_diagnostico']].count()

In [ ]:
tabela_is = casos.groupby(['se_is'])[['data_diagnostico']].count()
for row in tabela_is.index.values:
    if row.year < 2020:
        tabela_is = tabela_is.drop(row)

In [ ]:
tabela = pd.DataFrame(sivep['Positividade (%)']).rename(columns={'Positividade (%)': 'Positividade SRAG'})
notifica = pd.DataFrame(notifica['Positividade (%)']).rename(columns={'Positividade (%)': 'Positividade SG'})
notifica = notifica.reset_index()
tabela = tabela.reset_index()
tabela = tabela.merge(how = 'left', on = 'SE', right = notifica)

In [ ]:
tabela_diag = tabela_diag.rename(columns={'data_diagnostico': 'Diagnóstico'})
tabela_is = tabela_is.rename(columns={'data_diagnostico':'Primeiros sintomas'})

In [ ]:
index_list = pd.Series(dtype = 'int64')
for line in tabela_diag.index.values:
    index_list = index_list.append(pd.Series(line.cdcformat(), dtype = 'int64'))
tabela_diag = tabela_diag.set_index(index_list)
tabela_diag = tabela_diag.append(pd.DataFrame(tabela_diag.sum(0)).transpose())
tabela_diag = tabela_diag.reset_index()
tabela_diag = tabela_diag.rename(columns={'index':'SE'})
tabela = tabela.merge(how='left', on='SE', right = tabela_diag)

In [ ]:
index_list = pd.Series(dtype = 'int64')
for line in tabela_is.index.values:
    index_list = index_list.append(pd.Series(line.cdcformat(), dtype = 'int64'))
tabela_is = tabela_is.set_index(index_list)
tabela_is = tabela_is.append(pd.DataFrame(tabela_is.sum(0)).transpose())
tabela_is = tabela_is.reset_index()
tabela_is = tabela_is.rename(columns={'index':'SE'})
tabela = tabela.merge(how = 'left', on = 'SE', right = tabela_is)

In [ ]:
tabela['SE'] = tabela['SE'].astype('str').iloc[:-1].apply(lambda x: str(Week.fromstring(x).week))
tabela['SE'].iloc[-1] = 'Total Geral'
tabela = tabela.set_index('SE')
tabela.index.name = 'Semana Epidemiológica'

In [ ]:
tabela = tabela[['Primeiros sintomas', 'Diagnóstico', 'Positividade SG', 'Positividade SRAG']]

In [ ]:
tabela.to_excel(join(default_output, 'semanais', 'TABELA_apresentacao.xlsx'))

In [ ]:
tabela